In [284]:
from os import path

data_folder = "data"

#test = open(test_file, "r")
#print(test.readlines())

import os
for file in os.listdir("data"):
    if file.endswith(".txt"):
        print(file)

In [285]:
count = 0

segment_endings = []

with open(test_file, "r") as open_file:
    for line in open_file:
        check = line[:2]
        if check == "--" or check == "__":
            print("PAGE SEPERATION FOUND on line " + str(count))
            segment_endings.append(count)
        count = count + 1
        


PAGE SEPERATION FOUND on line 59
PAGE SEPERATION FOUND on line 60
PAGE SEPERATION FOUND on line 69
PAGE SEPERATION FOUND on line 81
PAGE SEPERATION FOUND on line 108
PAGE SEPERATION FOUND on line 109


In [286]:
segments = []

last_ending = -1

with open(test_file, "r") as open_file:
    lines = open_file.readlines()
    for ending in segment_endings:
        segment = lines[last_ending + 1:ending]
        segments.append(segment)
        last_ending = ending

In [287]:
rapport_page = segments[2]

print (rapport_page)

['\n', '\n', 'Rapport nummer: TDC1511.046\n', 'SAGOBAS-nummer: o151766g\n', '\n', 'Analyseresultaten van de olie uit de\n', 'transformator Pauwels 7840225\n', '\n']


In [288]:
rapport_nummer = ""
serie_nummer = ""

for line in rapport_page:
    if "Rapport nummer:" in line:
        rapport_nummer = line.strip()[16:]
    elif "transformator" in line:
        serie_nummer = ' '.join(line.split()[2:])

In [289]:
meta_page = segments[3]

In [290]:
bemonsterings_datum = ""

for line in meta_page:
    if "Bemonsteringsdatum" in line:
        bemonsterings_datum = line[-9:]
        
print(bemonsterings_datum)

15-11-17



In [291]:
gas_olie_page = segments[4]

conclusie_gas = ""
oordeel_gas = ""

for line in gas_olie_page:
    if line[0] == '-':
        conclusie_gas = conclusie_gas + line[2:-2]
    if conclusie_gas == "een normaal verouderingsproces":
        oordeel_gas = "+"
    elif conclusie_gas == "lekkage vanuit het lastschakelaarcompartiment":
        oordeel_gas = "+"
    elif conclusie_gas == "geen meting beschikbaar volgens normaal regime":
        oordeel_gas = "0"
    elif conclusie_gas == "wisselwerking tussen olie en verzinkte bak":
        oordeel_gas = "0"
    elif conclusie_gas == "thermische fout":
        oordeel_gas = "-"
    elif conclusie_gas == "dielektrische fout":
        oordeel_gas = "-"
    elif conclusie_gas == "deelontladingen":
        oordeel_gas = "-"
    elif conclusie_gas == "thermische belasting van het papier":
        oordeel_gas = "-"
    elif conclusie_gas == "ontladingen":
        oordeel_gas = "-"
    elif conclusie_gas == "vervolgstudie/einde levensduur starten":
        oordeel_gas = "-"

In [292]:
resterende_rapporten = segments[5:7]

In [293]:
def fysische_eigenschappen(page):
    blocks = split_in_blocks(page)
    return ' '.join(blocks[2]).strip()

In [294]:
def furanen(page):
    blocks = split_in_blocks(page)
    return ' '.join(blocks[2][1:]).strip()

In [295]:
def split_in_blocks(lines):
    block_separators = []
    count = 0
    for line in lines:
        if line == '\n':
            block_separators.append(count)
        count = count + 1
        
    blocks = []
    previous_ending = -1
    for separator in block_separators:
        block = lines[previous_ending+1:separator]
        previous_ending = separator
        if block:
            blocks.append(block)

    return(blocks)

In [296]:
import re

conclusie_fysische_eigenschappen = ""
oordeel_fysishe_eigenschappen = ""
conclusie_furanen = ""
oordeel_furanen = ""

for page in resterende_rapporten:
    for line in page:
        if "Eigenschappen van de olie" in line:
            conclusie_fysische_eigenschappen = fysische_eigenschappen(page)
            break
        if "Furanen" in line:
            conclusie_furanen = furanen(page)
            break
            
if conclusie_fysische_eigenschappen == "De olie voldoet aan IEC 60422 en kan in bedrijf blijven.":
    oordeel_fysische_eigenschappen = "+"
elif conclusie_fysische_eigenschappen == "Geen meting beschikbaar volgens normaal regime.":
    oordeel_fysische_eigenschappen = "0"
elif conclusie_fysische_eigenschappen == "":
    oordeel_fysische_eigenschappen = ""
else:
    oordeel_fysische_eigenschappen = "-"

dp_waarde = re.findall('\d+', conclusie_furanen)

if dp_waarde == ['550']:
    oordeel_furanen = "+"
elif dp_waarde == ['350']:
    oordeel_furanen = "+"
elif dp_waarde == ['250']:
    oordeel_furanen = "-"

In [297]:
print("rapport nummer: " + rapport_nummer)
print("serie nummer: " + serie_nummer)
print("olie in gas: " + conclusie_gas)
print("oordeel OIG: " + oordeel_gas)
print("fysische eigenschappen: " + conclusie_fysische_eigenschappen)
print("oordeel FE: " + oordeel_fysische_eigenschappen)
print("furanen: " + conclusie_furanen)
print("oordeel F: " + oordeel_furanen)

rapport nummer: TDC1511.046
serie nummer: 7840225
olie in gas: lekkage vanuit het lastschakelaarcompartiment
oordeel OIG: +
fysische eigenschappen: 
oordeel FE: 
furanen: 
oordeel F: 


In [298]:
row = {"rapport_nummer": rapport_nummer, "serie_nummer": serie_nummer, "olie in gas": conclusie_gas, "oordeel OIG": oordeel_gas, "fysische eigenschappen": conclusie_fysische_eigenschappen, "oordeel FE": oordeel_fysische_eigenschappen, "furanen": conclusie_furanen, "oordeel F": oordeel_furanen}

In [299]:
print(row)

{'oordeel FE': '', 'oordeel OIG': '+', 'olie in gas': 'lekkage vanuit het lastschakelaarcompartiment', 'furanen': '', 'rapport_nummer': 'TDC1511.046', 'serie_nummer': '7840225', 'oordeel F': '', 'fysische eigenschappen': ''}
